In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [12]:
VGG_types= {
    'VGG11':[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13':[64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16':[64, 64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512, 512,512, 'M', 512, 512, 512, 'M'],
    'VGG19':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}


Then flatten and 4096 X 4096 X 1000 Linear Layers

In [14]:
class VGG_Net(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(VGG_Net, self).__init__()
        self.in_channels= in_channels
        self.conv_layers= self.create_conv_layers(VGG_types['VGG11']) #change model per your requirement here.
        self.fcs= nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p= 0.5),
            nn.Linear(4096, num_classes)
            )
        
    def forward(self, x):
        x= self.conv_layers(x)
        x= x.reshape(x.shape[0], -1)
        x= self.fcs(x)
        return x
    
    def create_conv_layers(self, architecture):
        layers= []
        in_channels= self.in_channels
        
        for x in architecture:
            if type(x)== int:
                out_channels= x
                
                layers += [nn.Conv2d(in_channels= in_channels, out_channels= out_channels,
                                    kernel_size=(3, 3),
                                     stride= (1, 1), padding= (1, 1)),
                                     nn.BatchNorm2d(x),
                                     nn.ReLU()]
                in_channels= x
            elif x== 'M':
                layers += [nn.MaxPool2d(kernel_size= (2, 2), stride= (2, 2))]
        return nn.Sequential(*layers)

device= "cude" if torch.cuda.is_available() else 'cpu'
model= VGG_Net(in_channels= 3, num_classes= 1000)
x= torch.randn(1, 3, 224, 224)
print(model(x).shape)

torch.Size([1, 1000])


In [5]:
224/(2**5)

7.0